In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
#export
from exp.imports import *

In [73]:
#export
from typing import Union
from pathlib import Path
import zipfile


import gzip
import shutil


# Download Data sets

In [4]:
#export
URL ='https://data.4tu.nl/repository/'

## URL Class

In [42]:
#export

class URLs():
    "Global constants for dataset URLs."
    LOCAL_PATH = Path.cwd()


    
    # BPI Challenges
    BPIC_2012  = f'{URL}uuid:3926db30-f712-4394-aebc-75976070e91f/DATA1'
    
    BPIC_2013_closed_problems  = 'https://data.4tu.nl/repository/uuid:c2c3b154-ab26-4b31-a0e8-8f2350ddac11/DATA1?download=true'
    BPIC_2013_open_problems = 'https://data.4tu.nl/repository/uuid:3537c19d-6c64-4b1d-815d-915ab0e479da/DATA1?download=true'
    BPIC_2013_incidents = 'https://data.4tu.nl/repository/uuid:500573e6-accc-4b0c-9576-aa5468b10cee/DATA1?download=true'
   
    BPIC_2014_incidents_activity_log = 'https://data.4tu.nl/repository/uuid:86977bac-f874-49cf-8337-80f26bf5d2ef/DATA?download=true'
    BPIC_2014_change_details = 'https://data.4tu.nl/repository/uuid:d5ccb355-ca67-480f-8739-289b9b593aaf/DATA?download=true'
    BPIC_2014_incident_details = 'https://data.4tu.nl/repository/uuid:3cfa2260-f5c5-44be-afe1-b70d35288d6d/DATA?download=true'
    BPIC_2014_interaction_details = 'https://data.4tu.nl/repository/uuid:3d5ae0ce-198c-4b5c-b0f9-60d3035d07bf/DATA?download=true'
   
    BPIC_2015_Municipality_1 = 'https://data.4tu.nl/repository/uuid:a0addfda-2044-4541-a450-fdcc9fe16d17/DATA1?download=true'
    BPIC_2015_Municipality_2 = 'https://data.4tu.nl/repository/uuid:63a8435a-077d-4ece-97cd-2c76d394d99c/DATA1?download=true'
    BPIC_2015_Municipality_3 = 'https://data.4tu.nl/repository/uuid:ed445cdd-27d5-4d77-a1f7-59fe7360cfbe/DATA1?download=true'
    BPIC_2015_Municipality_4 = 'https://data.4tu.nl/repository/uuid:679b11cf-47cd-459e-a6de-9ca614e25985/DATA1?download=true'
    BPIC_2015_Municipality_5 = 'https://data.4tu.nl/repository/uuid:b32c6fe5-f212-4286-9774-58dd53511cf8/DATA1?download=true'
    
    BPIC_2016_Clicks_Logged_in = 'https://data.4tu.nl/repository/uuid:01345ac4-7d1d-426e-92b8-24933a079412/DATA1?download=true'
    BPIC_2016_Clicks_Not_Logged_in = 'https://data.4tu.nl/repository/uuid:9b99a146-51b5-48df-aa70-288a76c82ec4/DATA1?download=true'
    BPIC_2016_Complaints = 'https://data.4tu.nl/repository/uuid:e30ba0c8-0039-4835-a493-6e3aa2301d3f/DATA?download=true'
    BPIC_2016_Questions = 'https://data.4tu.nl/repository/uuid:2b02709f-9a84-4538-a76a-eb002eacf8d1/DATA?download=true'
    BPIC_2016_Werkmap_Messages = 'https://data.4tu.nl/repository/uuid:c3f3ba2d-e81e-4274-87c7-882fa1dbab0d/DATA?download=true'
   
    BPIC_2017 = 'https://data.4tu.nl/repository/uuid:5f3067df-f10b-45da-b98b-86ae4c7a310b/DATA1'
    BPIC_2017_offer_log = 'https://data.4tu.nl/repository/uuid:7e326e7e-8b93-4701-8860-71213edf0fbe/DATA1'
    BPIC_2018 = 'https://data.4tu.nl/repository/uuid:3301445f-95e8-4ff0-98a4-901f1f204972/DATA1'
    BPIC_2019 = 'http://icpmconference.org/2019/wp-content/uploads/sites/6/2019/01/log_IEEE.xes_.gz'
    
    
    # Process Discovery Contest
    PDC_2016_Training = 'https://www.win.tue.nl/ieeetfpm/lib/exe/fetch.php?media=shared:downloads:contest_dataset.zip'
    PDC_2016_Test = 'https://www.win.tue.nl/ieeetfpm/lib/exe/fetch.php?media=shared:contest_testeventlog.zip'
    PDC_2016_Models = 'https://www.win.tue.nl/ieeetfpm/lib/exe/fetch.php?media=shared:contest_models.zip'
    
    
    PDC_2017_Training = 'https://www.win.tue.nl/ieeetfpm/lib/exe/fetch.php?media=shared:dc_traning_logs_2017.zip'
    PDC_2017_Test =  'https://www.win.tue.nl/ieeetfpm/lib/exe/fetch.php?media=shared:test_log_contest_2017.zip'
    PDC_2017_Models = 'https://www.win.tue.nl/ieeetfpm/lib/exe/fetch.php?media=shared:models_contest_2017.zip'
    
    
    
    PDC_2019_Training = 'http://icpmconference.org/2019/wp-content/uploads/sites/6/2019/01/PDCtrainingLogs.zip'
    PDC_2019_Test1 = 'http://icpmconference.org/2019/wp-content/uploads/sites/6/2019/04/logFebruary.zip'
    PDC_2019_Test2 = 'http://icpmconference.org/2019/wp-content/uploads/sites/6/2019/04/log-15March.zip'
    

In [62]:
#export
_filenames={
    # BPI Challenge
    URLs.BPIC_2012:"BPIC_2012/BPI_Challenge_2012.xes.gz",
    URLs.BPIC_2013_closed_problems: "BPIC_2013/BPI_Challenge_2013_closed_problems.xes.gz",
    URLs.BPIC_2013_open_problems: "BPIC_2013/BPI_Challenge_2013_open_problems.xes.gz",
    URLs.BPIC_2013_incidents: "BPIC_2013/BPI_Challenge_2013_incidents.xes.gz",
    URLs.BPIC_2014_incidents_activity_log: "BPIC_2014/Detail_Incident_Activity.csv",
    URLs.BPIC_2014_change_details: "BPIC_2014/Detail_Change.csv",
    URLs.BPIC_2014_incident_details: "BPIC_2014/Detail_Incident.csv",
    URLs.BPIC_2014_interaction_details: "BPIC_2014/Detail_Interaction.csv",
    URLs.BPIC_2015_Municipality_1: "BPIC_2015/BPIC15_1.xes",
    URLs.BPIC_2015_Municipality_2: "BPIC_2015/BPIC15_2.xes",
    URLs.BPIC_2015_Municipality_3: "BPIC_2015/BPIC15_3.xes",
    URLs.BPIC_2015_Municipality_4: "BPIC_2015/BPIC15_4.xes",
    URLs.BPIC_2015_Municipality_5: "BPIC_2015/BPIC15_5.xes",
    URLs.BPIC_2016_Clicks_Logged_in: "BPIC_2016/BPI2016_Clicks_Logged_In.csv.zip",
    URLs.BPIC_2016_Clicks_Not_Logged_in: "BPIC_2016/BPI2016_Clicks_NOT_Logged_In.csv.zip",
    URLs.BPIC_2016_Complaints: "BPIC_2016/BPI2016_Complaints.csv",
    URLs.BPIC_2016_Questions: "BPIC_2016/BPI2016_Questions.csv",
    URLs.BPIC_2016_Werkmap_Messages: "BPIC_2016/BPI2016_Werkmap_Messages.csv",
    URLs.BPIC_2017: "BPIC_2017/BPI_Challenge_2017.xes.gz",
    URLs.BPIC_2017_offer_log: "BPIC_2017/BPI_Challenge_2017_Offerlog.xes.gz",
    URLs.BPIC_2018: "BPIC_2018/BPI_Challenge_2018.xes",
    URLs.BPIC_2019: "BPIC_2019/BPI_Challenge_2019.xes.gz",
    
    # Process Discovery Contest
    
    URLs.PDC_2016_Training : 'PDC_2016/Training/Training.zip',
    URLs.PDC_2016_Test : 'PDC_2016/Test/Test.zip',
    URLs.PDC_2016_Models : 'PDC_2016/Models/Models.zip',
    
    URLs.PDC_2017_Training : 'PDC_2017/Training/Training.zip',
    URLs.PDC_2017_Test : 'PDC_2017/Test/Test.zip',
    URLs.PDC_2017_Models : 'PDC_2017/Models/Models.zip',
    
    URLs.PDC_2019_Training : 'PDC_2019/Training/Training.zip',
    URLs.PDC_2019_Test1 : 'PDC_2019/Test1/Test1.zip',
    URLs.PDC_2019_Test2 : 'PDC_2019/Test2/Test2.zip',
   
}

## Config Class

In [7]:
#export

class Config():
    "Creates a default config file 'config.yml' in $FASTPM HOME (default `~/.fastpm/`)"
    DEFAULT_CONFIG_LOCATION = os.path.expanduser(os.getenv('FASTPM_HOME', '~/.fastpm'))
    DEFAULT_CONFIG_PATH = DEFAULT_CONFIG_LOCATION + '/config.yml'
    DEFAULT_CONFIG = {
        'data_path': DEFAULT_CONFIG_LOCATION + '/data',
        'data_archive_path': DEFAULT_CONFIG_LOCATION + '/data'
    }

    @classmethod
    def get_key(cls, key):
        cf=cls.get()
        "Get the path to `key` in the config file."
        return cf.get(key, cls.DEFAULT_CONFIG.get(key,None))

    @classmethod
    def get_path(cls, path):
        "Get the `path` in the config file."
        return _expand_path(cls.get_key(path))

    @classmethod
    def data_path(cls):
        "Get the path to data in the config file."
        return cls.get_path('data_path')

    @classmethod
    def data_archive_path(cls):
        "Get the path to data archives in the config file."
        return cls.get_path('data_archive_path')


    @classmethod
    def get(cls, fpath=None, create_missing=True):
        "Retrieve the `Config` in `fpath`."
        fpath = _expand_path(fpath or cls.DEFAULT_CONFIG_PATH)
        if not fpath.exists() and create_missing: cls.create(fpath)
        assert fpath.exists(), f'Could not find config at: {fpath}. Please create'
        with open(fpath, 'r') as yaml_file:
            yfile=yaml.safe_load(yaml_file)
            return yfile

    @classmethod
    def create(cls, fpath):
        "Creates a `Config` from `fpath`."
        fpath = _expand_path(fpath)
        assert(fpath.suffix == '.yml')
        if fpath.exists(): return
        fpath.parent.mkdir(parents=True, exist_ok=True)
        with open(fpath, 'w') as yaml_file:
            yaml.dump(cls.DEFAULT_CONFIG, yaml_file, default_flow_style=False)


# Store Data sets on Disc

In [54]:
#export
def _expand_path(fpath): return Path(fpath).expanduser()
def url2name(url):
    fname=''
    if url in _filenames:
        fname=_filenames[url]
        print(fname)
    else: 
        u = requests.get(url)
        if "Content-Disposition" in u.headers.keys():
            fname = re.findall("filename=(.+)", u.headers["Content-Disposition"])[0].replace('"', '').replace("'", '').replace(";", '')
        else: fname = url.split("/")[-1]
    return fname 


#TODO: simplify this mess
def url2path(url, data=True):
    "Change `url` to a path."
    name = url2name(url)
    return datapath4file(name, archive=False) 


def datapath4file(filename, archive=True):
    "Return data path to `filename`, checking locally first then in the config file."
    local_path = URLs.LOCAL_PATH/'data'/filename
    if local_path.exists(): return local_path
    elif archive: return Config.data_archive_path() / filename
    else: return Config.data_path() / filename

def download_data(url:str, fname:PathOrStr=None) -> Path:
    "Download `url` to destination `fname`."
    fname = Path(ifnone(fname, url2path(url)))
    os.makedirs(fname.parent, exist_ok=True)
    if not fname.exists():
        print(f'Downloading {url}')
        download_url(url, fname)
    return fname



In [ ]:
download_data(URLs.BPIC_2012)

In [ ]:
download_data(URLs.BPIC_2013_closed_problems)
download_data(URLs.BPIC_2013_incidents)
download_data(URLs.BPIC_2013_open_problems)

In [ ]:
download_data(URLs.BPIC_2014_change_details)
download_data(URLs.BPIC_2014_incident_details)
download_data(URLs.BPIC_2014_incidents_activity_log)
download_data(URLs.BPIC_2014_interaction_details)


In [ ]:
download_data(URLs.BPIC_2015_Municipality_1)
download_data(URLs.BPIC_2015_Municipality_2)
download_data(URLs.BPIC_2015_Municipality_3)
download_data(URLs.BPIC_2015_Municipality_4)
download_data(URLs.BPIC_2015_Municipality_5)

In [ ]:
download_data(URLs.BPIC_2016_Clicks_Logged_in)
download_data(URLs.BPIC_2016_Clicks_Not_Logged_in)
download_data(URLs.BPIC_2016_Complaints)
download_data(URLs.BPIC_2016_Questions)
download_data(URLs.BPIC_2016_Werkmap_Messages)

In [ ]:
download_data(URLs.BPIC_2017)
download_data(URLs.BPIC_2017_offer_log)

In [ ]:
download_data(URLs.BPIC_2012)

In [ ]:
download_data(URLs.BPIC_2017)

In [ ]:
download_data(URLs.BPIC_2018)

In [ ]:
download_data(URLs.BPIC_2019)

# Untar multiple formats

In [67]:
#export
PathOrStr = Union[Path,str]

def unzip(path):
    newpath=path.parent
    if not newpath.exists():
        print('unzip ...')
        with zipfile.ZipFile(path, 'r') as zip_ref:
            zip_ref.extractall(path.parent)
    return newpath

def gunzip(path):
    newpath=path.parent/path.stem
    if not newpath.exists():
        print('gunzip ...')
        with gzip.open(path, 'rb') as f_in:
            with open(newpath, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
    return newpath

_sup_compressions = {
    '.zip': unzip,
    '.gz': gunzip,
}
def untar_data(url:str, fname:PathOrStr=None) -> Path:
    path=download_data(url,fname)
    extract = _sup_compressions.get(path.suffix, False)
    if not extract: 
        print('not supported compression format')
        return path
    return extract(path)
#untar_data(URLs.BPIC_2016_Clicks_Logged_in)

In [37]:
untar_data(URLs.BPIC_2017_offer_log)

PosixPath('/home/lahann/.fastpm/data/BPIC_2017/BPI_Challenge_2017_Offerlog.xes')

In [24]:

Path('BPIC_2013/BPI_Challenge_2013_closed_problems.xes.gz').parent/Path('BPIC_2013/BPI_Challenge_2013_closed_problems.xes.gz').stem


PosixPath('BPIC_2013/BPI_Challenge_2013_closed_problems.xes')

In [41]:
untar_data(URLs.BPIC_2012)

gunzip ...


PosixPath('/home/lahann/.fastpm/data/BPIC_2012/BPI_Challenge_2012.xes')

In [68]:
untar_data(URLs.PDC_2016_Training)

PDC_2016/Training/Training.zip


PosixPath('/home/lahann/.fastpm/data/PDC_2016/Training')

In [74]:
! /home/lahann/anaconda3/envs/fastpm/bin/python notebook2script.py 01_fastpm_datasets.ipynb

Converted 01_fastpm_datasets.ipynb to exp/datasets.py


In [1]:
import pandas as pd

In [6]:
%%timeit -n 5
df = pd.DataFrame(columns=["A", "B", "C"]); new_row = pd.Series({"A": 4, "B": 4, "C": 4})
for i in range(160):
    df.loc[i] = new_row

127 ms ± 3.2 ms per loop (mean ± std. dev. of 7 runs, 5 loops each)


In [9]:
%%timeit -n 5

l=[]
for i in range(160):
    l.append([4,4,4])
pd.DataFrame(l,columns=["A", "B", "C"]);

789 µs ± 298 µs per loop (mean ± std. dev. of 7 runs, 5 loops each)


In [17]:
%%timeit -n 5

df = pd.DataFrame(columns=["A", "B", "C"]); new_row = pd.Series({"A": 4, "B": 4, "C": 4})
for i in range(160):
    df = df.append(new_row, ignore_index=True)

# 6.59 s ± 53.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


82.2 ms ± 4.05 ms per loop (mean ± std. dev. of 7 runs, 5 loops each)
